In [2]:
# A simple demo to process the compiled dataset
# after the whole process, the files in original_compiled_data can be transformed to the files in data4training_model
# we have already prepared the processed files for data4training_model

In [28]:
import pandas as pd
import numpy as np
import pickle

pd.options.display.max_columns = None

# the address to read files in original_compiled_data folder in the "data" folder
prefix='./data/original_compiled_data/'

# chemical feature information of involved drugs (including ECFP and molecular graphs)
in_file=open(prefix+'drugcomb_alldruginfo_dict.pickle','rb')
drugcomb_alldruginfo_dict=pickle.load(in_file)
in_file.close()

# therapeutic synergy score (TE) samples
drugcomb=pd.read_csv(prefix+'drugcomb.csv')

# adverse effect (AE) samples
twosides=pd.read_csv(prefix+'twosides.csv')

# drug-target interaction (DTI) samples
drug_target=pd.read_csv(prefix+'drug_target_inter.csv')

# protein-protein interaction (PPI) samples
target_target=pd.read_csv(prefix+'target_target_inter.csv')

drugcomb=drugcomb.rename(columns={'drug_row':'drug1','drug_col':'drug2'})
twosides=twosides[['drug1','drug2','Polypharmacy Side Effect','Side Effect Name','drug1_lower','drug2_lower','unified_name']]

# check synergy scores with None value
# zip：None
drugcomb['synergy_zip']=drugcomb['synergy_zip'].astype('float')
print(drugcomb['synergy_zip'].max(),drugcomb['synergy_zip'].min(),drugcomb['synergy_zip'].mean(),drugcomb['synergy_zip'].median())

# hsa: None
drugcomb['synergy_hsa']=drugcomb['synergy_hsa'].astype('float')
print(drugcomb['synergy_hsa'].max(),drugcomb['synergy_hsa'].min(),drugcomb['synergy_hsa'].mean(),drugcomb['synergy_hsa'].median())

# bliss: None
drugcomb['synergy_bliss']=drugcomb['synergy_bliss'].astype('float')
print(drugcomb['synergy_bliss'].max(),drugcomb['synergy_bliss'].min(),drugcomb['synergy_bliss'].mean(),drugcomb['synergy_bliss'].median())

# loewe: 4
temp=drugcomb[drugcomb['synergy_loewe']!='\\N']
temp_drugset=set.union(set(temp['drug1']),set(temp['drug2']))
# remove these 4 samples
print(len(temp_drugset))
drugcomb=drugcomb.drop(index=(drugcomb.loc[(drugcomb['synergy_loewe']=='\\N')].index))
drugcomb['synergy_loewe']=drugcomb['synergy_loewe'].astype('float')
print(drugcomb['synergy_loewe'].max(),drugcomb['synergy_loewe'].min(),drugcomb['synergy_loewe'].mean(),drugcomb['synergy_loewe'].median())

128.433573916656 -86.1689137467893 -1.1658416418422064 -0.461670602840447
87.93870313788229 -90.13426667492 -1.228943063125428 -1.0785883625221
207.197249044782 -95.0502059205476 -1.3513368723624046 -0.543827700211486
213
82.9982087530354 -101.379902323728 -8.991664263881768 -4.3698299642188


In [29]:
# 处理完synergy score数据以后，首先需要给所有药物靶点定一个顺序，以生成邻接矩阵
drugset=list(set.union(set(drugcomb['drug1_lower']),set(drugcomb['drug2_lower'])))
# 给drugset使用sort确定一个固定顺序
drugset.sort()
drug2id_dict={drug: i for i,drug in enumerate(drugset)}

# 再生成target的顺序
drug_target=drug_target.rename(columns={'target':'gene symbol'})
drug_target=drug_target[['drug','drug_lower','gene symbol']]
temp=set(drug_target['gene symbol'])

# remove the wrong sample
target_target=target_target[['gene1 symbol','gene2 symbol']]
target_target.iloc[103608]['gene1 symbol']='WTIP'
targetset=list(set.union(set(target_target['gene1 symbol']),set(target_target['gene2 symbol'])))
# 给targetset使用sort确定一个固定顺序
targetset.sort()

# further check data quality
temp_2=set([target.lower() for target in temp])
targetset_2=set([target.lower() for target in targetset])
print(temp-set(targetset))
print(temp_2-targetset_2)
print(len(temp),len(temp_2),len(targetset),len(targetset_2))
temp_3=pd.DataFrame([target.lower() for target in targetset])
print(temp_3.iloc[temp_3.duplicated()[temp_3.duplicated()==True].index])
target_target[(target_target['gene1 symbol'].str.lower() == 'wtip') | (target_target['gene2 symbol'].str.lower() == 'wtip')]

# 将drug_target中独有的target也并入targetset
targetset=list(set.union(set(targetset),temp))
# 重新固定顺序
targetset.sort()
target2id_dict={target: i for i,target in enumerate(targetset)}

{'ACCN2', 'GPR19', 'ABP1', 'ALPPL2', 'CA13', 'ACCN1'}
{'gpr19', 'ca13', 'alppl2', 'abp1', 'accn2', 'accn1'}
825 825 12796 12796
Empty DataFrame
Columns: [0]
Index: []


In [30]:
print(len(set(drugcomb['unified_name'])),len(set(twosides['unified_name'])))
# 将drugcomb样本缩减至2764个，以与twosides对应
twosides_pair_set=set(twosides['unified_name'])
drugcomb_pair_set=set(drugcomb['unified_name'])
inter_pair_set=list(set.intersection(twosides_pair_set,drugcomb_pair_set))
# 将inter_pair_set顺序固定，这就可以减少一定的随机程度
inter_pair_set.sort()

# 将te数据集中所包含的总pair数量变为2764
drugcomb_reduced=[]
for row in np.array(drugcomb):
    if(row[-1] in inter_pair_set):
        drugcomb_reduced.append(row)
# 代替之前读取的drugcomb
drugcomb=pd.DataFrame(drugcomb_reduced,columns=drugcomb.columns)
print(len(set(drugcomb['unified_name'])),len(set(twosides['unified_name'])))

2837 2764


2764 2764


In [31]:
# 现在开始基于上面的数据重新生成drugcomb和twosides，以及根据这两个数据集生成新的drug-target和target-target数据
# 同时也会生成新的drugset和targetset

# 开始尝试削减cell line数量
# 先基于所有的cell line生成一个顺序字典
cellline_set=list(set(drugcomb['cell_line_name']))
# 固定set顺序
cellline_set.sort()

cellline_num_list=[]
for cellline in cellline_set:
    num=drugcomb[drugcomb['cell_line_name']==cellline].shape[0]
    cellline_num_list.append([cellline,num])
cellline_num_list=pd.DataFrame(cellline_num_list)
# cellline_num_list是一个包含每个cellline所对应药物对数的list
cellline_num_list = cellline_num_list.sort_values([1],ascending=(False)).reset_index(drop=True)

7089 18392
2694


In [32]:
# 根据20个cell line去得到对应的drug_comb样本
# obtain the samples which have the same number as the description in the manuscript
selected_cellline_num=20
drugcomb_sorted=[]
for _,cellline in cellline_num_list[:selected_cellline_num].iterrows():
    drugcomb_sorted.append(drugcomb[drugcomb['cell_line_name']==cellline[0]])
drugcomb_sorted=pd.concat(drugcomb_sorted,axis=0)
drugpair_sorted=set(drugcomb_sorted['unified_name'])
print(drugcomb_sorted.shape[0],drugcomb.shape[0])
print(len(drugpair_sorted))
# print(cellline_num_list[:selected_cellline_num])

7089 18392
2694


In [ ]:
# 当选择cell line总数设置为20时，大概drugcomb总样本数缩减至7000，约为本来的1/3
# twosides原总样本数大概为19000, 而twosides样本数据基本未发生变化: 19317 to 18832
# 同时，总drug pair数下降至2694，药物总数下降至159
drugcomb_sorted=drugcomb_sorted.reset_index(drop=True)
twosides_sorted=[]
for _, row in twosides.iterrows():
    if row['unified_name'] in drugpair_sorted:
        twosides_sorted.append(row)
twosides_sorted=pd.DataFrame(twosides_sorted,columns=twosides.columns).reset_index(drop=True)

print(len(set.union(set(drugcomb_sorted['drug1_lower']),set(drugcomb_sorted['drug2_lower']))))
print(len(set.union(set(twosides_sorted['drug1_lower']),set(twosides_sorted['drug2_lower']))))
print(len(set(twosides_sorted['unified_name'])))

In [34]:
# 基于新生成的drugcomb和twosides去构建新的drugset和targetset
# 同时需要处理drug-target以及target-target数据
drugset=list(set.union(set(drugcomb_sorted['drug1_lower']),set(drugcomb_sorted['drug2_lower'])))
# 用于给drugset使用sort确定一个固定顺序
drugset.sort()
drug2id_dict={drug: i for i,drug in enumerate(drugset)}

drug_target_sorted=[]
for row in np.array(drug_target):
    if row[1] in drugset:
        drug_target_sorted.append(row)
drug_target_sorted=pd.DataFrame(drug_target_sorted,columns=drug_target.columns)

targetset=set(drug_target_sorted['gene symbol'])
target_target_sorted=[]
for row in np.array(target_target):
    # 根据drug_target筛选出对应的target_target
    if ((row[0] in targetset) or (row[1] in targetset)):
        target_target_sorted.append(row)
target_target_sorted=pd.DataFrame(target_target_sorted,columns=target_target.columns)

targetset=list(set.union(targetset, set.union(set(target_target_sorted['gene1 symbol']),set(target_target_sorted['gene2 symbol']))))
targetset.sort()
target2id_dict={target: i for i,target in enumerate(targetset)}

print(drug_target_sorted.shape, target_target_sorted.shape)

(3920, 3) (100552, 2)


In [35]:
# 根据drug编号生成药物之间的邻接矩阵
# 非常重要：
# 因为可能需要使药物-药物对样本加倍，以使A-B,B-A预测结果相同，但在此处仅按照数据集中顺序进行
# 为防止数据泄露，仅考虑在训练时对样本数量进行加倍（也就是对除了药物-药物对之外的数据进行对角线对称操作）

# 在HNEMA中，drug-drug-interaction和target-target-interaction矩阵在对角线位置的值为0（但两者都是对角线对称矩阵A=A.T）
duplicated_te=[]
drug_te_drug_matrix=np.zeros((len(drug2id_dict),len(drug2id_dict)))
for _,row in drugcomb_sorted.iterrows():
    # row
    temp1=drug2id_dict[row['drug1_lower']]
    # col
    temp2=drug2id_dict[row['drug2_lower']]
    # 检查是否有A-B/B-A情况
    if(drug_te_drug_matrix[temp2,temp1]==1):
        duplicated_te.append(row)
    drug_te_drug_matrix[temp1,temp2]=1

# 从这里发现，其实在两个数据库(drugcomb/twosides)中原始的drug name的大小写也是不同的，所以要用drug_lower
duplicated_se=[]
drug_se_drug_matrix=np.zeros((len(drug2id_dict),len(drug2id_dict)))
for _,row in twosides_sorted.iterrows():
    # row
    temp1=drug2id_dict[row['drug1_lower']]
    # col
    temp2=drug2id_dict[row['drug2_lower']]
    # 检查是否有A-B/B-A情况
    if(drug_se_drug_matrix[temp2,temp1]==1):
        duplicated_se.append(row)
    drug_se_drug_matrix[temp1,temp2]=1

# te样本是多于se的
print(drug_se_drug_matrix.sum(),drug_te_drug_matrix.sum())
print('total unified name number in twosides:',len(set(drugcomb_sorted['unified_name'])))
# 无A-B/B-A情况
print(duplicated_te,duplicated_se)

2694.0 2694.0
total unified name number in twosides: 2694
[] []


In [ ]:
# for pre-processing DD (TE relationship) meta-paths

# A p-value, or probability value, is a number describing how likely it is that your data would have occurred by random chance (i.e. that the null hypothesis is true).
# A p-value less than 0.05 (typically ≤ 0.05) is statistically significant. It indicates strong evidence against the null hypothesis, as there is less than a 5% probability the null is correct (and the results are random). Therefore, we reject the null hypothesis, and accept the alternative hypothesis.

# PPF of the standard normal distribution for the probability 1-a = 0.95
# scipy.stats.norm.ppf(0.95)=1.64, https://educationalresearchtechniques.com/2018/09/24/z-scores-and-inferential-stats-in-python/
duplicated_te_new=[]
synergy_score_qualified=[]
drug_te_drug_matrix_new=np.zeros((len(drug2id_dict),len(drug2id_dict)))
used_synergy_score='synergy_loewe'
mean=np.mean(drugcomb_sorted[used_synergy_score])
std=np.std(drugcomb_sorted[used_synergy_score])
for _,row in drugcomb_sorted.iterrows():
    # row
    temp1=drug2id_dict[row['drug1_lower']]
    # col
    temp2=drug2id_dict[row['drug2_lower']]
    z_score=(float(row[used_synergy_score]) - mean) / std
    # 检查是否有A-B/B-A情况
    if(drug_te_drug_matrix_new[temp2,temp1]==1):
        duplicated_te_new.append(row)
    if z_score>=1.64:
        drug_te_drug_matrix_new[temp1,temp2]=1
        synergy_score_qualified.append(float(row[used_synergy_score]))
print(duplicated_te_new)
# 这样相当于阈值变为了20.95
print(len(synergy_score_qualified),np.max(synergy_score_qualified),np.min(synergy_score_qualified),np.mean(synergy_score_qualified))
# 大概原来百分之十的药物对存在te边（193/2694）

print(drug_te_drug_matrix_new.sum())
# 此时的drug_te_drug_matrix_new是不对称的

In [37]:
# 生成完drug-te-drug matrix以后再生成target-target interaction（生成对称矩阵）
target_target_matrix=np.zeros((len(target2id_dict),len(target2id_dict)))
for row in np.array(target_target_sorted):
    # row
    temp1=target2id_dict[row[0]]
    # col
    temp2=target2id_dict[row[1]]
    target_target_matrix[temp1,temp2]=1
    target_target_matrix[temp2,temp1]=1
    
# 再生成drug-target矩阵(drug用小写dict转换，gene用大写dict转换)
drug_target_matrix=np.zeros((len(drug2id_dict),len(target2id_dict)))
for row in np.array(drug_target_sorted):
    # drug
    temp1=drug2id_dict[row[1]]
    # target
    temp2=target2id_dict[row[2]]
    drug_target_matrix[temp1,temp2]=1

In [38]:
# 此时共得到两个drug-drug（不对称）,一个drug-target（不对称），一个target-target（对称）matrices

# 接下来处理drug SMILES子图数据
drugcomb_alldruginfo_dict_lower={}
for key in drugcomb_alldruginfo_dict.keys():
    drugcomb_alldruginfo_dict_lower[key.lower()]=drugcomb_alldruginfo_dict[key]

In [39]:
# total number of every type of samples
drugcomb_sorted.shape, twosides_sorted.shape, drug_target_sorted.shape, target_target_sorted.shape, drug_target_matrix.shape

((7089, 29), (18832, 7), (3920, 3), (100552, 2), (159, 12575))

In [40]:
# total number of drug pairs
len(set(drugcomb_sorted['unified_name'])), len(set(twosides_sorted['unified_name']))

(2694, 2694)

In [41]:
# the root for storing the files generated by the following code for model training (data4training_model folder in the "data" folder)
prefix_sorted='./data/data4training_model'

In [42]:
# 将上述矩阵切换为稀疏矩阵
from scipy import sparse
# 这些矩阵的顺序都是有drug2id以及target2id的dict决定的
drug_se_drug_coomatrix=sparse.coo_matrix(drug_se_drug_matrix)
drug_te_drug_coomatrix=sparse.coo_matrix(drug_te_drug_matrix)
drug_target_coomatrix=sparse.coo_matrix(drug_target_matrix)
target_target_coomatrix=sparse.coo_matrix(target_target_matrix)

# 存储新的te矩阵
drug_te_drug_coomatrix_new=sparse.coo_matrix(drug_te_drug_matrix_new)
# sparse.save_npz(prefix_sorted+'drug_te_drug_coomatrix_new.npz',drug_te_drug_coomatrix_new)
# sparse.save_npz(prefix_sorted+'drug_se_drug_coomatrix.npz',drug_se_drug_coomatrix)
# sparse.save_npz(prefix_sorted+'drug_te_drug_coomatrix.npz',drug_te_drug_coomatrix)
# sparse.save_npz(prefix_sorted+'drug_target_coomatrix.npz',drug_target_coomatrix)
# sparse.save_npz(prefix_sorted+'target_target_coomatrix.npz',target_target_coomatrix)

In [15]:
# drugcomb_sorted.to_csv(prefix_sorted+'drugcomb.csv',index=False)
# twosides_sorted.to_csv(prefix_sorted+'twosides.csv',index=False)
# drug_target_sorted.to_csv(prefix_sorted+'drug_target.csv',index=False)
# target_target_sorted.to_csv(prefix_sorted+'target_target.csv',index=False)

In [16]:
with open(prefix_sorted+'drug2id_dict.pickle','wb') as out_file:
    pickle.dump(drug2id_dict,out_file)
with open(prefix_sorted+'target2id_dict.pickle','wb') as out_file:
    pickle.dump(target2id_dict,out_file)

In [73]:
# for generating molecular graphs for the variant HNE-GIN

id2drug_dict={value: key for key,value in drug2id_dict.items()}
drug_graph_edges=[]
drug_graph_properties=[]
# 储存node feature
drug_graph_nodes=[]
drugid2morgan={}
for drugid in id2drug_dict.keys():
    # drugid是当前drug id
    druginfo=drugcomb_alldruginfo_dict_lower[id2drug_dict[drugid]]
    drugid2morgan[drugid]=druginfo['morgan_bit']
    counter=-1
    # 得到矩阵的某一行
    for row in druginfo['adjacent_matrix']:
        counter+=1
        # 循环矩阵某一行的非零元素的所在位置
        for dst in row.nonzero()[0]:
            drug_graph_edges.append([drugid,counter,dst])
    # 再加入自环
    temp=druginfo['adjacent_matrix'].shape[0]
    self_loop=[[drugid,i,i] for i in range(temp)]
    drug_graph_edges.extend(self_loop)
    # 其实在properties中，graph_id就是label（也就是药物标签）
    drug_graph_properties.append([drugid,drugid,temp])

    # 直接保留原本原子的原子数
    for atom in druginfo['atom_num']:
        drug_graph_nodes.append([drugid,atom])

drug_graph_edges=pd.DataFrame(drug_graph_edges,columns=['graph_id','src','dst'])
drug_graph_properties=pd.DataFrame(drug_graph_properties,columns=['graph_id','label','num_nodes'])
drug_graph_nodes=pd.DataFrame(drug_graph_nodes,columns=['graph_id','atom_num'])

In [ ]:
# 统计一下出现的原子个数集合
atom_set=list(set(drug_graph_nodes['atom_num']))
atom_set.sort()
id_transform={atom: i for i,atom in enumerate(atom_set)}
print(id_transform)

# a small test
atom_embedding=torch.nn.Embedding(len(atom_set),64)
atom_rel_list=[id_transform[i] for i in atom_set]
print(atom_rel_list)
atom_embedding=atom_embedding(torch.LongTensor(atom_rel_list))
print(atom_embedding.shape)
print(torch.LongTensor(atom_rel_list).size())

# 存储atomnum2id_dict,用于在模型中生成原子的one-hot特征
with open(prefix_sorted+'atomnum2id_dict.pickle','wb') as out_file:
    pickle.dump(id_transform,out_file)

drug_graph_edges.to_csv(prefix_sorted+'drug_graph_edges.csv',index=0)
drug_graph_properties.to_csv(prefix_sorted+'drug_graph_properties.csv',index=0)
drug_graph_nodes.to_csv(prefix_sorted+'drug_graph_nodes.csv',index=0)

In [45]:
# ******************************************************
# start to generate the data which is suitable for HNEMA
# ******************************************************

# fix the random seed
import random
random_seed = 1012
random.seed(random_seed)
np.random.seed(random_seed)
pd.set_option('display.max_columns', None)

In [46]:
# 此时te和se中drug_pair数已经相同
drug_te_drug_matrix.sum(), drug_se_drug_matrix.sum()

(2694.0, 2694.0)

In [48]:
# drug_target
drug_target_drugid=list(map(lambda n: drug2id_dict[n], list(drug_target_sorted['drug_lower'])))
drug_target_targetid=list(map(lambda n: target2id_dict[n], list(drug_target_sorted['gene symbol'])))
drug_target_sorted['drugid']=drug_target_drugid
drug_target_sorted['targetid']=drug_target_targetid

# target_target
target_target_targetid1=list(map(lambda n: target2id_dict[n], list(target_target_sorted['gene1 symbol'])))
target_target_targetid2=list(map(lambda n: target2id_dict[n], list(target_target_sorted['gene2 symbol'])))
target_target_sorted['targetid1']=target_target_targetid1
target_target_sorted['targetid2']=target_target_targetid2

# drug_se_drug
# ***********************************************************************************
# in this place, we also generated another type of meta-path, DD (AE/SE relationship)
# but we do not use it in current version of HNEMA
# ***********************************************************************************
drug_se_drug_drugid1=list(map(lambda n: drug2id_dict[n], list(twosides_sorted['drug1_lower'])))
drug_se_drug_drugid2=list(map(lambda n: drug2id_dict[n], list(twosides_sorted['drug2_lower'])))
twosides_sorted['drugid1']=drug_se_drug_drugid1
twosides_sorted['drugid2']=drug_se_drug_drugid2

# drug_te_drug
drug_te_drug_drugid1=list(map(lambda n: drug2id_dict[n], list(drugcomb_sorted['drug1_lower'])))
drug_te_drug_drugid2=list(map(lambda n: drug2id_dict[n], list(drugcomb_sorted['drug2_lower'])))
drugcomb_sorted['drugid1']=drug_te_drug_drugid1
drugcomb_sorted['drugid2']=drug_te_drug_drugid2

# 这里drug_se_drug和drug_te_drug是分开的,在选择完作为训练集的drug-drug pair序号以后，将两个dataframe合并成一个dataframe(用drug序号表示)

In [49]:
from copy import deepcopy

twosides_pair_set=set(twosides_sorted['unified_name'])
drugcomb_pair_set=set(drugcomb_sorted['unified_name'])
inter_pair_set=list(set.intersection(twosides_pair_set,drugcomb_pair_set))
inter_pair_set.sort()

folds=10
val_fold=8
test_fold=9

def data_split_for_training(folds,val_fold,test_fold):
    if val_fold<test_fold:
        del_list=[val_fold,test_fold]
    else:
        del_list=[test_fold,val_fold]

    prng = np.random.RandomState(random_seed)
    allindex = prng.permutation(len(inter_pair_set))
    pos_inter_fold = np.array_split(allindex, folds)

    val_pos_sample = pos_inter_fold[val_fold]
    test_pos_sample = pos_inter_fold[test_fold]
    train_pos_sample = deepcopy(pos_inter_fold)
    train_pos_sample.pop(del_list[0])
    train_pos_sample.pop(del_list[1]-1)
    train_pos_sample = np.concatenate(train_pos_sample)
    return np.array(sorted(train_pos_sample)), np.array(sorted(val_pos_sample)), np.array(sorted(test_pos_sample))

train_idx, val_idx, test_idx = data_split_for_training(folds,val_fold,test_fold)   
    
drugpair2id_dict={drugpair: i for i,drugpair in enumerate(inter_pair_set)}

# keys的顺序就是对应从0到最后一个，由于在生成训练集测试集时，index已经进行了打乱，在这里不需要进一步打乱
all_drugpair_name=np.array(list(drugpair2id_dict.keys()))
train_drugpair_name=all_drugpair_name[train_idx]
val_drugpair_name=all_drugpair_name[val_idx]
test_drugpair_name=all_drugpair_name[test_idx]

In [51]:
# check again
len(inter_pair_set), train_idx.shape[0]+val_idx.shape[0]+test_idx.shape[0]

(2694, 2694)

In [52]:
# 根据划分的出的pair确定相关的样本
drugcomb_train,drugcomb_val,drugcomb_test=[],[],[]
for _,row in drugcomb_sorted.iterrows():
    if(row['unified_name'] in set(train_drugpair_name)):
        drugcomb_train.append(row)
    if(row['unified_name'] in set(val_drugpair_name)):
        drugcomb_val.append(row)
    if(row['unified_name'] in set(test_drugpair_name)):
        drugcomb_test.append(row)

drugcomb_train=pd.DataFrame(drugcomb_train,columns=drugcomb_sorted.columns).reset_index(drop=True)
drugcomb_val=pd.DataFrame(drugcomb_val,columns=drugcomb_sorted.columns).reset_index(drop=True)
drugcomb_test=pd.DataFrame(drugcomb_test,columns=drugcomb_sorted.columns).reset_index(drop=True)

In [53]:
drugcomb_train=drugcomb_train.sort_values(['drugid1','drugid2'],ascending=(True,True)).reset_index(drop=True)
drugcomb_val=drugcomb_val.sort_values(['drugid1','drugid2'],ascending=(True,True)).reset_index(drop=True)
drugcomb_test=drugcomb_test.sort_values(['drugid1','drugid2'],ascending=(True,True)).reset_index(drop=True)

In [54]:
twosides_train,twosides_val,twosides_test=[],[],[]
for _,row in twosides_sorted.iterrows():
    if(row['unified_name'] in set(train_drugpair_name)):
        twosides_train.append(row)
    if(row['unified_name'] in set(val_drugpair_name)):
        twosides_val.append(row)
    if(row['unified_name'] in set(test_drugpair_name)):
        twosides_test.append(row)

twosides_train=pd.DataFrame(twosides_train,columns=twosides_sorted.columns).reset_index(drop=True)
twosides_val=pd.DataFrame(twosides_val,columns=twosides_sorted.columns).reset_index(drop=True)
twosides_test=pd.DataFrame(twosides_test,columns=twosides_sorted.columns).reset_index(drop=True)

In [55]:
twosides_train=twosides_train.sort_values(['drugid1','drugid2'],ascending=(True,True)).reset_index(drop=True)
twosides_val=twosides_val.sort_values(['drugid1','drugid2'],ascending=(True,True)).reset_index(drop=True)
twosides_test=twosides_test.sort_values(['drugid1','drugid2'],ascending=(True,True)).reset_index(drop=True)

In [56]:
# start to build the adjacency matrix for heterogeneous network embedding
cellline_set=list(set(drugcomb_sorted['cell_line_name']))
cellline_set.sort()

cellline2id_dict={cellline: i for i,cellline in enumerate(cellline_set)}
# with open(prefix+'cellline2id_dict.pickle','wb') as out_file:
    # pickle.dump(cellline2id_dict,out_file)
    
# the node order: 0 for drug, 1 for target, 2 for cell line
num_drug=len(drug2id_dict)
num_target=len(target2id_dict)
num_cellline=len(cellline2id_dict)
dim=num_drug+num_target+num_cellline

# give a type mask to every node in the dataset (because there are four node types)
type_mask=np.zeros((dim),dtype=int)
type_mask[num_drug:num_drug+num_target]=1
type_mask[num_drug+num_target:]=2

In [57]:
# 这一步需要生成邻接矩阵去存储异构网络中的所有关系，并且所有关系都是对称的
# 也就是说接下来生成的adjM是一个对角线对称矩阵，但其中验证集和测试集中的drug-drug pair需要去掉
# 而在之前所有drug-drug pair都不是对称的，以防止数据泄露，此时已经在训练集测试集分开
# 在这里可以考虑将drug-drug pair矩阵变为对称矩阵（参考DeepSynergy）
# 也就是说即使这里double了样本，也不会重复出现数据泄露，因为训练集和测试集基于不同的drug-pair进行了完全的区分

# test（证明当te和se矩阵对称时，两者是完全相同的，所以此时两者可以共享一个邻接矩阵区域，也就是drug-inter-drug区域）
drug_se_drug_matrix_symmetric=np.zeros(drug_se_drug_matrix.shape)
counter=-1
for row in np.array(drug_se_drug_matrix):
    counter+=1
    for col in row.nonzero()[0]:
        drug_se_drug_matrix_symmetric[col,counter]=1
drug_se_drug_matrix_symmetric+=np.array(drug_se_drug_matrix)
print((drug_se_drug_matrix_symmetric!=drug_se_drug_matrix_symmetric.T).sum())

drug_te_drug_matrix_symmetric=np.zeros(drug_te_drug_matrix.shape)
counter=-1
for row in np.array(drug_te_drug_matrix):
    counter+=1
    for col in row.nonzero()[0]:
        drug_te_drug_matrix_symmetric[col,counter]=1
drug_te_drug_matrix_symmetric+=np.array(drug_te_drug_matrix)
# print((drug_se_drug_matrix_symmetric!=drug_se_drug_matrix_symmetric.T).sum())
# print((drug_te_drug_matrix_symmetric!=drug_te_drug_matrix_symmetric.T).sum())
print((drug_se_drug_matrix_symmetric-drug_te_drug_matrix_symmetric).sum())

print(drug_te_drug_matrix_symmetric.sum(),drug_te_drug_matrix.sum(),drug_se_drug_matrix_symmetric.sum(),drug_se_drug_matrix.sum())
print('num_drug:',num_drug,'num_target:',num_target)

0
0.0
5388.0 2694.0 5388.0 2694.0
num_drug: 159 num_target: 12575


In [58]:
# 证明对称的se和te是一样之后，生成用于计算对称drug-drug矩阵的dataframe(此时的drug_drug并不是对称的)
# drug_drug来自于drugcomb_train,可用于生成drug_se_drug的关系
drug_drug_sorted=drugcomb_train[['drug1','drug2','drug1_lower','drug2_lower','drugid1','drugid2']]

# 但是drug-drug dataframe只用于生成drug-se-drug关系，还需要生成一个用于生成drug-te-drug关系(基于drug_drug)
# 但根据上面分析，这里只需生成训练集中的te关系即可
drug_te_drug_sorted=[]
for _,row in drug_drug_sorted.iterrows():
    temp1,temp2=row['drugid1'],row['drugid2']
    if ((drug_te_drug_matrix_new[temp1,temp2]==1) or (drug_te_drug_matrix_new[temp2,temp1]==1)):
        drug_te_drug_sorted.append(row)
        
# 共有211个样本（based on drug-drug-cell line pairs）> 193个总样本数（in drug_te_drug_matrix_new）
# 因为drugcomb_train中针对每个药物对可能会有多个cell line,但是对于d-te-d/d-se-d,只需保留一个综合结果即可
# 所以要在后面生成d-d metapath样本时，要进行去重操作
drug_te_drug_sorted=pd.DataFrame(drug_te_drug_sorted,columns=['drug1','drug2','drug1_lower','drug2_lower','drugid1','drugid2']).reset_index(drop=True)
print(drug_te_drug_sorted.shape)

(211, 6)


In [ ]:
# 这个邻接矩阵是中心对称的（例如将原来drug-target数据，变为了D-T,T-D两部分在adjM中）
adjM=np.zeros((dim,dim),dtype=int)

# store drug-target (0-1)
drug_target_sorted=drug_target_sorted.sort_values(['drugid','targetid'],ascending=(True,True)).reset_index(drop=True)
for _,row in drug_target_sorted.iterrows():
    uid=row['drugid']
    aid=num_drug+row['targetid']
    adjM[uid, aid] = 1
    adjM[aid, uid] = 1
    
# store target-target (1-1)
target_target_sorted=target_target_sorted.sort_values(['targetid1','targetid2'],ascending=(True,True)).reset_index(drop=True)
for _,row in target_target_sorted.iterrows():
    uid=num_drug+row['targetid1']
    aid=num_drug+row['targetid2']
    adjM[uid,aid]=1
    adjM[aid,uid]=1
    
adjM_te = deepcopy(adjM)
# store drug-se-drug (0-se-0)
# 存储的是训练集中的对称drug-drug pair (基于所有的te pair或所有的se pair)
for _,row in drug_drug_sorted.iterrows():
    uid=row['drugid1']
    aid=row['drugid2']
    adjM[uid,aid]=1
    adjM[aid,uid]=1
    
# store drug-te-drug (0-te-0)
# 存储的是训练集中的对称drug-drug pair (基于大于阈值的te pair)
for _,row in drug_te_drug_sorted.iterrows():
    uid=row['drugid1']
    aid=row['drugid2']
    adjM_te[uid,aid]=1
    adjM_te[aid,uid]=1

In [60]:
# ********************************************************************************************************
# obtain dicts storing the neighbors of different nodes with diferent types (based on different metapaths)
# ********************************************************************************************************

# metapath/edge types for drug embedding:
# 1.drug-target-drug (drug-target)
# 2.drug-target-target-drug (drug-target,target-target)
# 3.drug-target-target-target-drug (drug-target,target-target)
# 4.drug-se-drug (drug-drug) (finally it is not used)
# 5.drug-te-drug (drug-drug)

# indices that the dict stores are all based on the relative index instead of absolute index
# 也是就list中的关系都是用相对标签来储存

# 取得的就是每个drug所对应target的关系(可通过绘制adjM的对应区域来证明)
drug_target_list = {i: adjM[i, num_drug:].nonzero()[0] for i in range(num_drug)}
# 取得的就是每个target所对应drug的关系
target_drug_list = {
    i: adjM[i+num_drug, :num_drug].nonzero()[0] for i in range(num_target)}
# 取得的就是每个target所对应的target的关系(由于基于adjM,所以里面的结果是对称的)
target_target_list = {
    i: adjM[i+num_drug, num_drug:].nonzero()[0] for i in range(num_target)}
# 取得的就是每个drug所对应的drug的关系(基于se或者所有的te)
drug_drug_list = {
    i: adjM[i, :num_drug].nonzero()[0] for i in range(num_drug)}
# 取得的就是每个drug所对应的drug的关系(超过阈值的te)
drug_te_drug_list = {
    i: adjM_te[i, :num_drug].nonzero()[0] for i in range(num_drug)}

In [62]:
# generate different metapath-based datasets (based on absolute indices)

# metapath index uses absoluate index
# drug-target-drug (0-1-0)
drug_target_drug=[]
# 相比drug_target_list,target_drug_list会稀疏很多，因为只有小部分target与drug直接相连
for target, drug_list in target_drug_list.items():
    # the order of drug1 and drug2 should not be important
    drug_target_drug.extend([(drug1, target, drug2)
                            for drug1 in drug_list for drug2 in drug_list])
drug_target_drug = np.array(drug_target_drug)
# transform all the node indices to absolute indices
drug_target_drug[:, 1] += num_drug
sorted_index = sorted(list(range(len(drug_target_drug))),
                      key=lambda i: drug_target_drug[i, [0, 2, 1]].tolist())
# sort the drug_target_drug according to the priority order: drug1->drug2->target
# 也就是以两边的drug需要来进行排序
# 此时所包含的是绝对标签
drug_target_drug = drug_target_drug[sorted_index]
print('drug_target_drug.shape:',drug_target_drug.shape)

drug_target_drug.shape: (43906, 3)


In [49]:
# drug-target-target-drug (0-1-1-0)
# 先获得target-inter-target (1-1)
# drug1-target1-target2-drug2和drug2-target2-target1-drug1都是有意义的，并且这个关系只有在将target-target变成对称的情况下才能生成
# 否则只有target1-target2或target2-target1，上述两种情况分别用于生成drug2的embedding和drug1的embedding
target_inter_target=target_target_sorted[['targetid1','targetid2']].to_numpy(dtype=np.int32)
target_inter_target=np.concatenate([target_inter_target,target_inter_target[:,[1,0]]],axis=0)
target_inter_target=[tuple(row) for row in target_inter_target]
target_inter_target=np.unique(target_inter_target,axis=0)
# 在这里不需要变为绝对标签，因为下一步搜索所需的target_drug_list基于相对标签
# target_inter_target += num_drug
sorted_index = sorted(list(range(len(target_inter_target))),
                      key=lambda i: target_inter_target[i].tolist())
target_inter_target = target_inter_target[sorted_index]
print('target_inter_target.shape:',target_inter_target.shape)

target_inter_target.shape: (201104, 2)


In [50]:
# 再获得0-1-1-0
drug_target_target_drug=[]
for target1,target2 in target_inter_target:
     drug_target_target_drug.extend([(drug1, target1, target2, drug2) for drug1 in target_drug_list[target1] for drug2 in target_drug_list[target2]])
drug_target_target_drug=np.array(drug_target_target_drug)
#将相对标签转换为绝对标签
drug_target_target_drug[:, [1, 2]] += num_drug
sorted_index = sorted(list(range(len(drug_target_target_drug))), key=lambda i : drug_target_target_drug[i, [0, 3, 1, 2]].tolist())
drug_target_target_drug = drug_target_target_drug[sorted_index]
print('drug_target_target_drug.shape:',drug_target_target_drug.shape)

drug_target_target_drug.shape: (1620274, 4)


In [51]:
# drug-target-target-target-drug (0-1-1-1-0)
# 先获得target-target-target (1-1-1)
target_target_target=[]
for target,target_list in target_target_list.items():
    target_target_target.extend([(target1,target,target2) for target1 in target_list for target2 in target_list])
target_target_target=np.array(target_target_target)
# 目前没有将相对坐标转化成绝对坐标
# 因为在target_drug_list中使用的是相对坐标
sorted_index=sorted(list(range(len(target_target_target))),key=lambda i: target_target_target[i,[0,2,1]].tolist())
target_target_target=target_target_target[sorted_index]

In [52]:
print('target_target_target.shape:',target_target_target.shape)

target_target_target.shape: (39462620, 3)


In [53]:
drug_target_target_target_drug=[]
# this sampling ratio could be further adjusted according to the computational resources you have
dtttd_ratio=0.3
for target1,target,target2 in target_target_target:
    # target_drug_list使用相对坐标，目前target1,target,target2都是相对坐标
    if(len(target_drug_list[target1])==0 or len(target_drug_list[target])==0):
        continue
    candidate_drug1_list=np.random.choice(len(target_drug_list[target1]),
                                          int(dtttd_ratio*len(target_drug_list[target1])),replace=False)
    candidate_drug1_list=target_drug_list[target1][candidate_drug1_list]

    candidate_drug2_list=np.random.choice(len(target_drug_list[target2]),
                                         int(dtttd_ratio*len(target_drug_list[target2])),replace=False)
    candidate_drug2_list=target_drug_list[target2][candidate_drug2_list]
    
    drug_target_target_target_drug.extend([(drug1,target1,target,target2,drug2) for drug1 in candidate_drug1_list for drug2 in candidate_drug2_list])

drug_target_target_target_drug=np.array(drug_target_target_target_drug)
#将相对标签转换为绝对标签
drug_target_target_target_drug[:, [1, 2, 3]] += num_drug
sorted_index=sorted(list(range(len(drug_target_target_target_drug))),key=lambda i:drug_target_target_target_drug[i,[0,4,1,2,3]].tolist())
drug_target_target_target_drug=drug_target_target_target_drug[sorted_index]
print('drug_target_target_target_drug.shape:',drug_target_target_target_drug.shape)

drug_target_target_target_drug.shape: (7116096, 5)


In [63]:
# 然后处理drug-drug数据，在这里要注意只使用训练集中drug-drug pair作为样本
# 因为这些metapath用于训练模型,所以所有的drug-drug pair都是对称的
# 同0-1-1-0，有了drug1-drug2和drug2-drug1才能同时为drug1和drug2生成embedding

# 这里处理的是经过阈值筛选的te training子图
# metapath index for therapeutic effect
drug_inter1_drug=drug_te_drug_sorted[['drugid1','drugid2']].to_numpy(dtype=np.int32)
drug_inter1_drug=np.concatenate([drug_inter1_drug,drug_inter1_drug[:,[1,0]]],axis=0)
drug_inter1_drug=[tuple(row) for row in drug_inter1_drug]
drug_inter1_drug=np.unique(drug_inter1_drug,axis=0)

sorted_index = sorted(list(range(len(drug_inter1_drug))),
                      key=lambda i: drug_inter1_drug[i].tolist())
drug_inter1_drug = drug_inter1_drug[sorted_index]
print('drug_inter1_drug.shape:',drug_inter1_drug.shape)

drug_inter1_drug.shape: (294, 2)


In [55]:
# metapath index for side effect (it is not used in current HNEMA)
drug_inter2_drug=drug_drug_sorted[['drugid1','drugid2']].to_numpy(dtype=np.int32)
drug_inter2_drug=np.concatenate([drug_inter2_drug,drug_inter2_drug[:,[1,0]]],axis=0)
drug_inter2_drug=[tuple(row) for row in drug_inter2_drug]
drug_inter2_drug=np.unique(drug_inter2_drug,axis=0)

sorted_index = sorted(list(range(len(drug_inter2_drug))),
                      key=lambda i: drug_inter2_drug[i].tolist())
drug_inter2_drug = drug_inter2_drug[sorted_index]
print('drug_inter2_drug.shape:',drug_inter2_drug.shape)

drug_inter2_drug.shape: (4312, 2)


In [ ]:
# 在训练集某个batch中，上述两个集合中与要预测的pair重复的pair会被去掉（正负方向都会去掉）

In [ ]:
# start the saving process of generated meta-path instances
import pathlib

# 先基于所有的cell line生成一个顺序字典
cellline_set=list(set(drugcomb_sorted['cell_line_name']))
cellline_set.sort()
cellline2id_dict={cellline: i for i,cellline in enumerate(cellline_set)}
with open(prefix_sorted+'cellline2id_dict.pickle','wb') as out_file:
    pickle.dump(cellline2id_dict,out_file)

drugpair2id_dict={drugpair: i for i,drugpair in enumerate(inter_pair_set)}

involved_metapaths=[
    [(0,1,0),(0,1,1,0),(0,1,1,1,0),(0,'te',0),(0,'se',0)]
]

for i in range(len(involved_metapaths)):
    pathlib.Path(prefix_sorted+'{}'.format(i)).mkdir(parents=True, exist_ok=True)
    
metapath_indices_mapping = {
    (0, 1, 0): drug_target_drug,
    (0, 1, 1, 0): drug_target_target_drug,
    (0, 1, 1, 1, 0): drug_target_target_target_drug,
    (0, 'te', 0): drug_inter1_drug,
    (0, 'se', 0): drug_inter2_drug,
}

In [58]:
# store all data（按照metapath_indices_mapping逻辑储存即可）
# 实际上没有用上protein target标签，因为这里只生成了基于drug的metapath
target_idx_lists = [np.arange(num_drug), np.arange(num_target)]
offset_list = [0, num_drug]
for i, metapaths in enumerate(involved_metapaths):
    for metapath in metapaths:
        # get corresponding metapath nodes
        edge_metapath_idx_array = metapath_indices_mapping[metapath]
        
        # store the metapath node as pickle file
        with open(prefix_sorted+'{}/'.format(i)+'-'.join(map(str, metapath))+'_idx.pickle', 'wb') as out_file:
            target_metapaths_mapping={}
            left=0
            right=0
            for target_idx in target_idx_lists[i]:
                # target_idx refers a specific drug or target specified by an index (在这里也就是每个药物的id)
                # the aim is to locate the last position of a metapath of a node in edge_metapath_idx_array
                # edge_metapath_idx_array is ordered by the first node in a metapath
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                # first select all metapath choices using the first node, then put the first node into the last position of the metapath choice as the target node
                # edge_metapath_idx_array[left:right, ::-1]: reverse every metapath choice
                # and then store them to 'target_metapaths_mapping'
                # 也就是说edge_metapath_idx_array是按照第0个元素进行排序的，所以可通过left和right获得每个药物节点所对应的所有元路径
                # 在GCN中，最后一个节点是中心节点，也就是target节点，所以需要将每个metapath颠倒一下再放入
                target_metapaths_mapping[target_idx] = edge_metapath_idx_array[left:right, ::-1]
                # move to the next drug/target node
                left = right
            # write dict after iterations
            pickle.dump(target_metapaths_mapping, out_file)
        
        # store the corresponding source and target node of metapath choice as adjlist file
        # 再存储所有metapath中中心节点的邻居节点
        with open(prefix_sorted + '{}/'.format(i) + '-'.join(map(str, metapath)) + '.adjlist', 'w') as out_file:
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                # adjlist is based on relative index
                neighbors = edge_metapath_idx_array[left:right, -1] - \
                    offset_list[i]
                neighbors = list(map(str, neighbors))
                if(len(neighbors) > 0):
                    # write line in each iteration
                    # 格式： central node + neighboring nodes(可能包括他们自身)
                    # 例子： 9 9 9 15 28 69 154 184 332 343 449 477 543 544 549 567 570 574 654 669 669 683 (第一个9是central node)
                    out_file.write('{} '.format(target_idx) +
                                   ' '.join(neighbors) + '\n')
                else:
                    out_file.write('{}\n'.format(target_idx))
                left = right

In [61]:
import scipy
from scipy import sparse

# 此时验证集和测试集中的drug-drug pair还未加倍（也就是还没有生成drug pairs with the opposite drug order）
# drugcomb_train.to_csv(prefix_sorted+'drugcomb_train.csv',index=0)
# drugcomb_val.to_csv(prefix_sorted+'drugcomb_val.csv',index=0)
# drugcomb_test.to_csv(prefix_sorted+'drugcomb_test.csv',index=0)

# twosides_train.to_csv(prefix_sorted+'twosides_train.csv',index=0)
# twosides_val.to_csv(prefix_sorted+'twosides_val.csv',index=0)
# twosides_test.to_csv(prefix_sorted+'twosides_test.csv',index=0)

# store the heterogeneous adjacent matrix using a sparse npz file (which stores the model training data)
scipy.sparse.save_npz(prefix_sorted+'adjM.npz',
                      scipy.sparse.csr_matrix(adjM))
# store the node type mask of all nodes
np.save(prefix_sorted+'node_types.npy', type_mask)

# 在HNEMA中，最终需要重新生成一个完整的drug-target的numpy列表并储存，因为drug-target在操作过程中被替换为了训练数据

In [65]:
# start to generate each drug-drug-cell line pair with AE and TE labels/values

# 根据划分的出的pair确定相关的样本
drugcomb_train,drugcomb_val,drugcomb_test=[],[],[]

# drugcomb在缩减为2764条样本之后再没有被修改过
for _,row in drugcomb_sorted.iterrows():
    if(row['unified_name'] in set(train_drugpair_name)):
        drugcomb_train.append(row)
    if(row['unified_name'] in set(val_drugpair_name)):
        drugcomb_val.append(row)
    if(row['unified_name'] in set(test_drugpair_name)):
        drugcomb_test.append(row)

drugcomb_train=pd.DataFrame(drugcomb_train,columns=drugcomb_sorted.columns).reset_index(drop=True)
drugcomb_val=pd.DataFrame(drugcomb_val,columns=drugcomb_sorted.columns).reset_index(drop=True)
drugcomb_test=pd.DataFrame(drugcomb_test,columns=drugcomb_sorted.columns).reset_index(drop=True)

print(train_drugpair_name.shape[0],val_drugpair_name.shape[0],test_drugpair_name.shape[0])

2156 269 269


In [66]:
# 将训练集的样本量加倍
drugcomb_train=drugcomb_train[['drugid1','drugid2','cell_line_name','S_mean','synergy_zip','synergy_loewe','synergy_hsa','synergy_bliss']]
drugcomb_train_=drugcomb_train[['drugid2','drugid1','cell_line_name','S_mean','synergy_zip','synergy_loewe','synergy_hsa','synergy_bliss']]
drugcomb_train=pd.DataFrame(np.concatenate([np.array(drugcomb_train),np.array(drugcomb_train_)],axis=0),columns=drugcomb_train.columns)
drugcomb_train=drugcomb_train.sort_values(['drugid1', 'drugid2'],ascending=(True,True)).reset_index(drop=True)

# 在训练集中使样本数量翻倍，在验证集和测试集中保持不变
# 然后在模型的一个batch中，将样本翻倍并评估，而不是在这里
# 因为此时一个batch中每个样本都会有两个顺序，将测试集generator大小减半即可
drugcomb_val=drugcomb_val[['drugid1','drugid2','cell_line_name','S_mean','synergy_zip','synergy_loewe','synergy_hsa','synergy_bliss']]
# drugcomb_val_=drugcomb_val[['drugid2','drugid1','cell_line_name','S_mean','synergy_zip','synergy_loewe','synergy_hsa','synergy_bliss']]
# drugcomb_val=pd.DataFrame(np.concatenate([np.array(drugcomb_val),np.array(drugcomb_val_)],axis=0),columns=drugcomb_val.columns)
drugcomb_val=drugcomb_val.sort_values(['drugid1', 'drugid2'],ascending=(True,True)).reset_index(drop=True)

drugcomb_test=drugcomb_test[['drugid1','drugid2','cell_line_name','S_mean','synergy_zip','synergy_loewe','synergy_hsa','synergy_bliss']]
# drugcomb_test_=drugcomb_test[['drugid2','drugid1','cell_line_name','S_mean','synergy_zip','synergy_loewe','synergy_hsa','synergy_bliss']]
# drugcomb_test=pd.DataFrame(np.concatenate([np.array(drugcomb_test),np.array(drugcomb_test_)],axis=0),columns=drugcomb_test.columns)
drugcomb_test=drugcomb_test.sort_values(['drugid1', 'drugid2'],ascending=(True,True)).reset_index(drop=True)

In [67]:
drugcomb_train.shape,drugcomb_val.shape,drugcomb_test.shape

((11254, 8), (812, 8), (650, 8))

In [68]:
# 将所用样本和标签重新生成一次
twosides_train,twosides_val,twosides_test=[],[],[]
for _,row in twosides_sorted.iterrows():
    if(row['unified_name'] in set(train_drugpair_name)):
        twosides_train.append(row)
    if(row['unified_name'] in set(val_drugpair_name)):
        twosides_val.append(row)
    if(row['unified_name'] in set(test_drugpair_name)):
        twosides_test.append(row)

twosides_train=pd.DataFrame(twosides_train,columns=twosides_sorted.columns).reset_index(drop=True)
twosides_val=pd.DataFrame(twosides_val,columns=twosides_sorted.columns).reset_index(drop=True)
twosides_test=pd.DataFrame(twosides_test,columns=twosides_sorted.columns).reset_index(drop=True)

twosides_train=twosides_train[['drugid1','drugid2','Polypharmacy Side Effect','Side Effect Name']]
twosides_train_=twosides_train[['drugid2','drugid1','Polypharmacy Side Effect','Side Effect Name']]
twosides_train=pd.DataFrame(np.concatenate([np.array(twosides_train),np.array(twosides_train_)],axis=0),columns=twosides_train.columns)
twosides_train=twosides_train.sort_values(['drugid1', 'drugid2'],ascending=(True,True)).reset_index(drop=True)

twosides_val=twosides_val[['drugid1','drugid2','Polypharmacy Side Effect','Side Effect Name']]
# twosides_val_=twosides_val[['drugid2','drugid1','Polypharmacy Side Effect','Side Effect Name']]
# twosides_val=pd.DataFrame(np.concatenate([np.array(twosides_val),np.array(twosides_val_)],axis=0),columns=twosides_val.columns)
twosides_val=twosides_val.sort_values(['drugid1', 'drugid2'],ascending=(True,True)).reset_index(drop=True)

twosides_test=twosides_test[['drugid1','drugid2','Polypharmacy Side Effect','Side Effect Name']]
# twosides_test_=twosides_test[['drugid2','drugid1','Polypharmacy Side Effect','Side Effect Name']]
# twosides_test=pd.DataFrame(np.concatenate([np.array(twosides_test),np.array(twosides_test_)],axis=0),columns=twosides_test.columns)
twosides_test=twosides_test.sort_values(['drugid1', 'drugid2'],ascending=(True,True)).reset_index(drop=True)

In [69]:
# 合并drugcomb和twosides
# 首先生成twosides的标签顺序
se_symbolset=list(set(twosides_sorted['Polypharmacy Side Effect']))
se_symbolset.sort()
se_symbol2id_dict={se: i for i,se in enumerate(se_symbolset)}

with open(prefix_sorted+'se_symbol2id_dict.pickle','wb') as out_file:
    pickle.dump(se_symbol2id_dict,out_file)

# start to create adverse effect labels based on drugcomb/TE samples (as TE prediction is our main task)

# training
train_se_label=np.zeros((drugcomb_train.shape[0],len(se_symbol2id_dict)))
drugcomb_train_numpy=np.array(drugcomb_train)
twosides_train_numpy=np.array(twosides_train)
counter=-1
counter_=0
for row in np.array(drugcomb_train_numpy):
    counter+=1
    drugid1=row[0]
    drugid2=row[1]
    search=twosides_train_numpy[((twosides_train_numpy[:,0]==drugid1)&(twosides_train_numpy[:,1]==drugid2)) |
                                ((twosides_train_numpy[:,1]==drugid1)&(twosides_train_numpy[:,0]==drugid2))]
    search=[se_symbol2id_dict[i] for i in search[:,2]]
    counter_+=len(search)
    train_se_label[counter,search]=1

In [70]:
# validation
val_se_label=np.zeros((drugcomb_val.shape[0],len(se_symbol2id_dict)))
drugcomb_val_numpy=np.array(drugcomb_val)
twosides_val_numpy=np.array(twosides_val)
counter=-1
for row in np.array(drugcomb_val_numpy):
    counter+=1
    drugid1=row[0]
    drugid2=row[1]
    search=twosides_val_numpy[((twosides_val_numpy[:,0]==drugid1)&(twosides_val_numpy[:,1]==drugid2)) |
                            ((twosides_val_numpy[:,1]==drugid1)&(twosides_val_numpy[:,0]==drugid2))]
    search=[se_symbol2id_dict[i] for i in search[:,2]]
    val_se_label[counter,search]=1

In [ ]:
# test
test_se_label=np.zeros((drugcomb_test.shape[0],len(se_symbol2id_dict)))
drugcomb_test_numpy=np.array(drugcomb_test)
twosides_test_numpy=np.array(twosides_test)
counter=-1
for row in np.array(drugcomb_test_numpy):
    counter+=1
    drugid1=row[0]
    drugid2=row[1]
    search=twosides_test_numpy[((twosides_test_numpy[:,0]==drugid1)&(twosides_test_numpy[:,1]==drugid2)) |
                        ((twosides_test_numpy[:,1]==drugid1)&(twosides_test_numpy[:,0]==drugid2))]
    search=[se_symbol2id_dict[i] for i in search[:,2]]
    test_se_label[counter,search]=1

In [ ]:
# 基于drugcomb的样本建立te标签(顺序：S_mean，synergy_zip，synergy_loewe，synergy_hsa，synergy_bliss)
train_te_label=np.array(drugcomb_train)[:,3:]
val_te_label=np.array(drugcomb_val)[:,3:]
test_te_label=np.array(drugcomb_test)[:,3:]

# npy file can store one standard binary numpy matrix, while npz as a zip could store multiple npy(s)
np.savez(prefix_sorted+'train_val_test_drug_drug_samples.npz',
         train_drug_drug_samples=np.array(drugcomb_train,dtype=str)[:,:3],
         val_drug_drug_samples=np.array(drugcomb_val,dtype=str)[:,:3],
         test_drug_drug_samples=np.array(drugcomb_test,dtype=str)[:,:3])

np.savez(prefix_sorted+'train_val_test_drug_drug_labels.npz',
         train_te_labels=train_te_label.astype('float32'),
         train_se_labels=train_se_label.astype('float32'),
         val_te_labels=val_te_label.astype('float32'),
         val_se_labels=val_se_label.astype('float32'),
         test_te_labels=test_te_label.astype('float32'),
         test_se_labels=test_se_label.astype('float32'))

In [ ]:
# create ECFP6 of each drug
ECFP6_DNN=[]
for key in drugid2morgan.keys():
    ECFP6_DNN.append(drugid2morgan[key])
ECFP6_DNN=np.array(ECFP6_DNN)

ECFP6_DNN_coomatrix=sparse.coo_matrix(ECFP6_DNN)
sparse.save_npz(prefix_sorted+'ECFP6_DNN_coomatrix.npz',ECFP6_DNN_coomatrix)
print(ECFP6_DNN.shape)